## PiecewiseJerkPath Problem Formulate

该Task的相关内容：modules/planning/tasks/optimizers/piecewise_jerk_path

根据输入的参考线、上下边界，生成平滑路径

将路况（车道、障碍物）抽象为上下边界boundary

定义cost包括：平滑性、安全性（远离障碍物并尽量靠近车道中心）、以及终点状态

沿s方向均匀采样，假设相邻采样点之间的三阶导数（jerk）为常数，求解s-l/dl/ddl


目前还有几个问题：

w_ref中的使用高斯函数生成的权重

scale_factor的影响

jerk_bound

### qp的一般形式
$$
minimize \frac{1}{2} \cdot x^T \cdot P \cdot x + q \cdot x 
\\
s.t. LB \leq A\cdot x \leq UB
$$

### Cost
整理后，目前代码中的cost为：
$$
f(l(s)) = w_l\cdot \sum_{i=0}^{n-1} l_i^2 + w_{{l}'}\cdot \sum_{i=0}^{n-1} {l_i}'^2 + w_{{l}''}\cdot \sum_{i=0}^{n-1} {l_i}''^2 + w_{{l}'''}\cdot \sum_{i=0}^{n-2}(\frac{{l_{i+1}}'' - {l_i}''}{\Delta s})^2 \\
+w_{obs}\cdot \sum_{i=0}^{n-1}(l_i-l_{ref})^2 \\
+w_{end_l}\cdot (l_{n-1} - l_{endref})^2 + w_{end_{dl}}\cdot ({l}'_{n-1}-{l_{endref}}')^2 + w_{end_{ddl}}\cdot ({l}''_{n-1} - {l_{endref}}'')^2
$$

第一行代表平滑性权重 : 偏移量 + 一阶导 + 二阶导 + 三阶导

第二行表示远离障碍物的同时尽可能靠近中心线 ： ref = (上边界 + 下边界)/2

第三行表示终点状态的权重:终点状态使用cost表示，期望终点状态尽量贴近ref，但也要考虑平滑性和中心线；起点使用bound约束，保证轨迹从自车出发。

### 将cost转换为二次规划格式 - P和q矩阵
为了方便以6个点为例，
这部分的源代码的位置modules/planning/math/piecewise_jerk/piecewise_jerk_path_problem.cc：

#### cost - 平滑
$$
w_l\cdot \sum_{i=0}^{n-1} l_i^2 + w_{{l}'}\cdot \sum_{i=0}^{n-1} {l_i}'^2 + w_{{l}''}\cdot \sum_{i=0}^{n-1} {l_i}''^2 + w_{{l}'''}\cdot \sum_{i=0}^{n-2}(\frac{{l_{i+1}}'' - {l_i}''}{\Delta s})^2 \\
$$
转为P矩阵：

$$\begin{vmatrix}
w_l&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&w_l&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&w_l&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&w_l&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&w_l&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&w_l&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&w_{{l}'}&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&w_{{l}'}&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&w_{{l}'}&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&w_{{l}'}&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&w_{{l}'}&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&w_{{l}'}&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&w_{{l}''}+\frac{w_{{l}'''}}{\Delta s^2}&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&-2\frac{w_{{l}'''}}{\Delta s^2}&w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&-2\frac{w_{{l}'''}}{\Delta s^2}&w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&-2\frac{w_{{l}'''}}{\Delta s^2}&w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&-2\frac{w_{{l}'''}}{\Delta s^2}&w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&-2\frac{w_{{l}'''}}{\Delta s^2}&w_{{l}''}+\frac{w_{{l}'''}}{\Delta s^2}\\
\end{vmatrix}$$

#### cost - ref
$$
w_{obs}\cdot \sum_{i=0}^{n-1}(l_i-l_{ref})^2 \\
$$
P：
$$\begin{vmatrix}
w_{ref_1}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&w_{ref_2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&w_{ref_3}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&w_{ref_4}&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&w_{ref_5}&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&w_{ref_6}&0&0&0&0&0&0&0&0&0&0&0&0\\
\end{vmatrix}$$
q：
$$\begin{vmatrix}
-2w_{ref_1}\cdot l_{ref_1}\\
-2w_{ref_2}\cdot l_{ref_2}\\
-2w_{ref_3}\cdot l_{ref_3}\\
-2w_{ref_4}\cdot l_{ref_4}\\
-2w_{ref_5}\cdot l_{ref_5}\\
-2w_{ref_6}\cdot l_{ref_6}\\
\end{vmatrix}$$

#### cost - end_state
$$
w_{end_l}\cdot (l_{n-1} - l_{endref})^2 + w_{end_{dl}}\cdot ({l}'_{n-1}-{l_{endref}}')^2 + w_{end_{ddl}}\cdot ({l}''_{n-1} - {l_{endref}}'')^2
$$
P：
$$\begin{vmatrix}
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&w_{end_l}&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&w_{end_{dl}}&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&w_{end_{ddl}}\\
\end{vmatrix}$$
q:
$$\begin{vmatrix}
0\\
0\\
0\\
0\\
0\\
-2w_{end_l}\cdot end_l\\
0\\
0\\
0\\
0\\
0\\
-2w_{end_dl}\cdot end_dl\\
0\\
0\\
0\\
0\\
0\\
-2w_{end_ddl}\cdot end_ddl\\
\end{vmatrix}$$

#### combine
上面三组加在一起，就得到了我们需要的P矩阵和q矩阵

scale_factor：不同阶的放大系数

P矩阵：
$$\begin{vmatrix}
\frac{w_l+w_{ref_1}}{scale\_factor_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&\frac{w_l+w_{ref_2}}{scale\_factor_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&\frac{w_l+w_{ref_3}}{scale\_factor_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&\frac{w_l+w_{ref_4}}{scale\_factor_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&\frac{w_l+w_{ref_5}}{scale\_factor_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&\frac{w_l+w_{ref_6}+w_{end_l}}{scale\_factor_0^2}&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&\frac{w_{{l}'}}{scale\_factor_1^2}&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&\frac{w_{{l}'}}{scale\_factor_1^2}&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&\frac{w_{{l}'}}{scale\_factor_1^2}&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&\frac{w_{{l}'}}{scale\_factor_1^2}&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&\frac{w_{{l}'}}{scale\_factor_1^2}&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&\frac{w_{{l}'}+w_{end_{dl}}}{scale\_factor_1^2}&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&\frac{w_{{l}''}+\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&\frac{w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&\frac{w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&\frac{w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&\frac{w_{{l}''}+2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\frac{w_{{l}'''}}{\Delta s^2}}{scale\_factor_2^2}&\frac{w_{{l}''}+\frac{w_{{l}'''}}{\Delta s^2}+w_end_{ddl}}{scale\_factor_2^2}\\
\end{vmatrix}$$

q矩阵：

$$\begin{vmatrix}
-2w_{ref_1}\cdot l_{ref_1} / scale\_factor_0\\
-2w_{ref_2}\cdot l_{ref_2}/ scale\_factor_0\\
-2w_{ref_3}\cdot l_{ref_3}/ scale\_factor_0\\
-2w_{ref_4}\cdot l_{ref_4}/ scale\_factor_0\\
-2w_{ref_5}\cdot l_{ref_5}/ scale\_factor_0\\
-2w_{ref_6}\cdot l_{ref_6}-2w_{end_l}\cdot end_l/ scale\_factor_0\\
0\\
0\\
0\\
0\\
0\\
-2w_{end_{dl}}\cdot end_{dl}/ scale\_factor_1\\
0\\
0\\
0\\
0\\
0\\
-2w_{end_{ddl}}\cdot end_{ddl}/ scale\_factor_2\\
\end{vmatrix}$$


### Bound
Bound主要包括:

1.参考线边界：在Decider中获取的参考线上下边界（非凸问题转化为凸优化）

2.起点约束：以自车的当前状态为起点进行规划

3.数学约束：用于描述变量s-l，一阶导l',二阶导l'',三阶导l'''，之间的数学关系，在认为相邻点的三阶导l'''是常量的前提下，可得：

$$
{l_{i+1}}'' = \int_{0}^{\Delta s}{l_{i->i+1}}'''ds={l_i}''+{l_{i->i+1}}'''\cdot\Delta s\\
{l_{i+1}}'={l_i}'+\int_{0}^{\Delta s}{l}''(s)ds={l_i}'+{l_i}''\cdot\Delta s+\frac{1}{2}{l_{i->i+1}}'''\cdot\Delta s^2\\
l_{i+1}=l_i+\int_{0}^{\Delta s}{l}'(s)ds\\=l_i+(l_i)'\cdot\Delta s+\frac{1}{2}{l_i}''\cdot s^2+\frac{1}{6}{l_{i->i+1}}'''\cdot\Delta s^3
$$

### 将bound转换为二次规划格式 - LB和UB和A矩阵

1.道路边界：

\begin{split}\begin{array}
 & \begin{bmatrix}
lb_{s1} \\ 
lb_{s2} \\ 
lb_{s3} \\
lb_{s4} \\ 
lb_{s5} \\ 
lb_{s6} 
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
1&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&1&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&1&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&1&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&1&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&1&0&0&0&0&0&0&0&0&0&0&0&0\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
ub_{s1} \\ 
ub_{s2} \\ 
ub_{s3} \\
ub_{s4} \\ 
ub_{s5} \\ 
ub_{s6} 
 \end{bmatrix}
\end{array}\end{split}

2.dl限制：~tanΘ ， 目前取固定值±2.0
\begin{split}\begin{array}
 & \begin{bmatrix}
-2.0 \\ 
-2.0 \\ 
-2.0 \\
-2.0 \\ 
-2.0 \\ 
-2.0 
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
0&0&0&0&0&0&1&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&1&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&1&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&1&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&1&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&1&0&0&0&0&0&0\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
2.0 \\ 
2.0 \\ 
2.0 \\
2.0 \\ 
2.0 \\ 
2.0 
 \end{bmatrix}
\end{array}\end{split}

3.ddl限制：~kappa，在道路曲率的基础上，附加自车可以向左/右转的最大曲率，得到kappa_bound
\begin{split}\begin{array}
 & \begin{bmatrix}
-lat\_acc\_bound - kappa_{s1} \\ 
-lat\_acc\_bound - kappa_{s2} \\ 
-lat\_acc\_bound - kappa_{s3} \\
-lat\_acc\_bound - kappa_{s4} \\ 
-lat\_acc\_bound - kappa_{s5} \\ 
-lat\_acc\_bound - kappa_{s6} 
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
0&0&0&0&0&0&0&0&0&0&0&0&1&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&1&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&1&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&1&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&1&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&1\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
lat\_acc\_bound - kappa_{s1} \\ 
lat\_acc\_bound - kappa_{s2} \\ 
lat\_acc\_bound - kappa_{s3} \\
lat\_acc\_bound - kappa_{s4} \\ 
lat\_acc\_bound - kappa_{s5} \\ 
lat\_acc\_bound - kappa_{s6} 
 \end{bmatrix}
\end{array}\end{split}

4.起点约束
\begin{split}\begin{array}
 & \begin{bmatrix}
ego_l \\ 
ego_{dl} \\ 
ego_{ddl} \\
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
1&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&1&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&1&0&0&0&0&0\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
ego_l \\ 
ego_{dl} \\ 
ego_{ddl} \\
 \end{bmatrix}
\end{array}\end{split}

5.数学关系约束

5.1 dddl约束：
$$
{l_{i+1}}'' = \int_{0}^{\Delta s}{l_{i->i+1}}'''ds={l_i}''+{l_{i->i+1}}'''\cdot\Delta s\\
$$
即：
l(i->i+1)''' = (l(i+1)'' - l(i)'') / delta_s

jerk_bound = max_yaw_rate / axis_distance / vehicle_speed

转为矩阵形式：
\begin{split}\begin{array}
 & \begin{bmatrix}
-jerk_1 * \Delta s \\ 
-jerk_2 * \Delta s \\ 
-jerk_3 * \Delta s \\
-jerk_4 * \Delta s \\ 
-jerk_5 * \Delta s \\
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
0&0&0&0&0&0&0&0&0&0&0&0&-1&1&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&-1&1&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&-1&1&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&-1&1&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&-1&1\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
jerk_1 * \Delta s \\ 
jerk_2 * \Delta s \\ 
jerk_3 * \Delta s \\
jerk_4 * \Delta s \\ 
jerk_5 * \Delta s \\
 \end{bmatrix}
\end{array}\end{split}

5.2 ddl约束：
$$
{l_{i+1}}'={l_i}'+\int_{0}^{\Delta s}{l}''(s)ds={l_i}'+{l_i}''\cdot\Delta s+\frac{1}{2}{l_{i->i+1}}'''\cdot\Delta s^2\\
$$
将dddl式代入，可得 l(i+1)' - l(i)' - 0.5 * delta_s * l(i)'' - 0.5 * delta_s * l(i+1)'' = 0
转为矩阵形式：
\begin{split}\begin{array}
 & \begin{bmatrix}
0 \\ 
0 \\ 
0 \\
0 \\ 
0 \\
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
0&0&0&0&0&0&-1&1&0&0&0&0&-0.5\cdot\Delta s&-0.5\cdot\Delta s&0&0&0&0\\
0&0&0&0&0&0&0&-1&1&0&0&0&0&-0.5\cdot\Delta s&-0.5\cdot\Delta s&0&0&0\\
0&0&0&0&0&0&0&0&-1&1&0&0&0&0&-0.5\cdot\Delta s&-0.5\cdot\Delta s&0&0\\
0&0&0&0&0&0&0&0&0&-1&1&0&0&0&0&-0.5\cdot\Delta s&-0.5\cdot\Delta s&0\\
0&0&0&0&0&0&0&0&0&0&-1&1&0&0&0&0&-0.5\cdot\Delta s&-0.5\cdot\Delta s\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
0 \\ 
0 \\ 
0 \\
0 \\ 
0 \\
 \end{bmatrix}
\end{array}\end{split}

5.3 dl约束：
$$
l_{i+1}=l_i+\int_{0}^{\Delta s}{l}'(s)ds=l_i+(l_i)'\cdot\Delta s+\frac{1}{2}{l_i}''\cdot s^2+\frac{1}{6}{l_{i->i+1}}'''\cdot\Delta s^3
$$
将dddl式代入，可得：l(i+1) - l(i) - delta_s * l(i)' - 1/3 * delta_s^2 * l(i)'' - 1/6 * delta_s^2 * l(i+1)'' = 0

转为矩阵：
\begin{split}\begin{array}
 & \begin{bmatrix}
0 \\ 
0 \\ 
0 \\
0 \\ 
0 \\
\end{bmatrix} 
 \leq 
 \begin{bmatrix} 
-1&1&0&0&0&0&-\Delta s&0&0&0&0&0&-\frac{1}{3}\Delta s^2&-\frac{1}{6}\Delta s^2&0&0&0&0\\
0&-1&1&0&0&0&0&-\Delta s&0&0&0&0&0&-\frac{1}{3}\Delta s^2&-\frac{1}{6}\Delta s^2&0&0&0\\
0&0&-1&1&0&0&0&0&-\Delta s&0&0&0&0&0&-\frac{1}{3}\Delta s^2&-\frac{1}{6}\Delta s^2&0&0\\
0&0&0&-1&1&0&0&0&0&-\Delta s&0&0&0&0&0&-\frac{1}{3}\Delta s^2&-\frac{1}{6}\Delta s^2&0\\
0&0&0&0&-1&1&0&0&0&0&-\Delta s&0&0&0&0&0&-\frac{1}{3}\Delta s^2&-\frac{1}{6}\Delta s^2\\
 \end{bmatrix} 
 x 
 \leq  
 \begin{bmatrix}
0 \\ 
0 \\ 
0 \\
0 \\ 
0 \\
 \end{bmatrix}
\end{array}\end{split}

以上就是整个formulate problem的过程